In [95]:
import pymongo
import time
import threading
import random
from pymongo.collection import Collection

In [96]:
cl = pymongo.MongoClient("mongodb://localhost:27017")
db = cl["deep"]
obj1 = db["obj1"]
obj2 = db["obj2"]
obj4 = db["obj4"]
obj8 = db["obj8"]
coll_list = [obj1, obj2, obj4, obj8]

In [97]:
user_list = [1, 2, 4]
depth_list = [1, 2, 4, 8]

In [98]:
obj1.find_one()

{'_id': ObjectId('687a79485702990e0d27b649'),
 'l1': {'device': 5550, 'app': 'executive', 'total_volume_bytes': 3636}}

In [109]:
where_1 = {"l1.device": {"$gte": 9900}}
where_2 = {"l1.l2.device": {"$gte": 9900}}
where_4 = {"l1.l2.l3.l4.device": {"$gte": 9900}}
where_8 = {"l1.l2.l3.l4.l5.l6.l7.l8.device": {"$gte": 9890}}
where_list = [where_1, where_2, where_4, where_8]

In [110]:
for c in range(len(coll_list)):
    print(coll_list[c].count_documents(where_list[c]))

1037
1023
1041
1051


In [101]:
def resolve_where(depth: int):
    return where_list[depth]

In [83]:
def measure_select(user_id: int, duration_list: list[int], coll: Collection, whereClause: list[dict]):
    duration = 0
    tries = 1
    for i in range(tries):
        start = time.perf_counter()
        result = list(coll.find(whereClause))
        end = time.perf_counter()
        duration += end - start
    
    duration = (duration / tries) * 1000
    duration_list[user_id] = duration

In [21]:
def test_select(num_of_users, depth):
    threads_list = []
    duration_list = [None] * num_of_users
    whereClause= resolve_where(depth)
    coll = coll_list[depth]

    for user_id in range (num_of_users):
        t = threading.Thread(target=measure_select, args=(user_id, duration_list, coll, whereClause,))
        t.start()
        threads_list.append(t)

    for t in threads_list:
        t.join()
        
    result = sum(duration_list) / len(duration_list)
    return result    

In [34]:
def execute_select():
    u_len = len(user_list)
    d_len = len(depth_list)
    result = [[0 for _ in range(d_len)] for _ in range(u_len)]
    
    for u in range(u_len):
        print(f"user: {user_list[u]}")
        print("  depth:", end=" ")
        for d in range(d_len):
            result[u][d] = test_select(user_list[u], d)
            print(f"{depth_list[d]}", end=" ")

        print("---")
    return result

In [31]:
def calculate_diffs(old, new):
    for u in range(len(user_list)):
        print(f"user {user_list[u]}")
        for d in range(len(depth_list)):
            o_val = old[u][d]
            n_val = new[u][d]
            diff = ((n_val - o_val) / o_val) * 100
            print(f"| {depth_list[d]} |{o_val:10.0f}| {n_val:10.0f}| {diff:10.1f} |")
        print("--------")

In [24]:
def createIndexes():
    obj1.create_index([("l1.device", 1)])
    obj2.create_index([("l1.l2.device", 1)])
    obj4.create_index([("l1.l2.l3.l4.device", 1)])
    obj8.create_index([("l1.l2.l3.l4.l5.l6.l7.l8.device", 1)])

In [25]:
def clearIndexes():
    obj1.drop_indexes()
    obj2.drop_indexes()
    obj4.drop_indexes()
    obj8.drop_indexes()

In [26]:
def showIndexes():
    for i in obj1.list_indexes():
        print(i)
    for a in obj2.list_indexes():
        print(a)
    for b in obj4.list_indexes():
        print(b)
    for c in obj8.list_indexes():
        print(c)

In [111]:
clearIndexes()

In [112]:
showIndexes()

SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])
SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])
SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])
SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])


In [113]:
select_old = execute_select()

user: 1
  depth: 1 2 4 8 ---
user: 2
  depth: 1 2 4 8 ---
user: 4
  depth: 1 2 4 8 ---


In [114]:
createIndexes()

In [115]:
select_new = execute_select()

user: 1
  depth: 1 2 4 8 ---
user: 2
  depth: 1 2 4 8 ---
user: 4
  depth: 1 2 4 8 ---


In [116]:
calculate_diffs(select_old, select_new)

user 1
| 1 |        69|         10|      -86.2 |
| 2 |       194|         12|      -94.0 |
| 4 |        79|         17|      -78.7 |
| 8 |       163|         25|      -84.6 |
--------
user 2
| 1 |        91|         12|      -87.0 |
| 2 |        98|         12|      -87.5 |
| 4 |       110|         18|      -83.7 |
| 8 |       134|         21|      -84.0 |
--------
user 4
| 1 |       164|         16|      -90.1 |
| 2 |       173|         20|      -88.2 |
| 4 |       166|         30|      -82.1 |
| 8 |       217|        163|      -24.8 |
--------
